In [32]:
import pandas as pd
import os
import random
from tqdm import tqdm
import pickle
from sklearn.preprocessing import MinMaxScaler

In [33]:
regenerate_context_data = True
sequence_augmentation = True
whole_session_context = False
model_test_run = False
data_autoencoder = False
pad_to_window_size = True
feat_engg = True

PATH_TO_LOAD = '../data/04_Merged'
if feat_engg:
    combined_context_path = '../data/06_context_feat_engg/data_featue_engineering.csv'
    augmentation_folder = 'featengg/' if sequence_augmentation else 'non_aug/'
else:
    combined_context_path = '../data/05_Interaction_Sequences/context.csv'
    augmentation_folder = 'fix/' if sequence_augmentation else 'non_aug/'

window = 100 #seconds

base_path = '../datasets/sequential/'
if model_test_run:
    augmentation_folder = 'test/aug/' if sequence_augmentation else 'test/non_aug/'

sequence_context_path = f'{base_path}{augmentation_folder}parameters/sequence_context.csv'
parameter_path = f'{base_path}{augmentation_folder}parameters'
train_session_path = f'{base_path}{augmentation_folder}parameters/train_sessions.pkl'
test_session_path = f'{base_path}{augmentation_folder}parameters/test_sessions.pkl'
train_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/train.csv'
test_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/test.csv'
train_static_context_path = f'{base_path}{augmentation_folder}static_context/train.csv'
test_static_context_path = f'{base_path}{augmentation_folder}static_context/test.csv'
train_sequence_path = f'{base_path}{augmentation_folder}seq/train.tsv'
test_sequence_path = f'{base_path}{augmentation_folder}seq/test.tsv'
train_dense_static_context_path = f'{base_path}{augmentation_folder}dense_static_context/train.csv'
test_dense_static_context_path = f'{base_path}{augmentation_folder}dense_static_context/test.csv'

In [34]:
all_columns = ['index', 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'odometer', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'ESP_speed', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l', 'seatbelt_rear_m',
       'seatbelt_rear_r', 'CHA_ESP_drive_mode', 'CHA_MO_drive_mode',
       'rain_sensor', 'street_category', 'kickdown', 'altitude',
       'driving_program', 'datetime', 'session', 'Label', 'ID',
       'FunctionValue', 'domain', 'BeginTime', 'time_second',
       'distance_driven', 'ts_normalized', 'weekday']

selected = [ 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l',
       'seatbelt_rear_r', 'street_category', 'altitude',
       'datetime', 'session', 'time_second',
       'distance_driven', 'weekday'
]

bad_quality = ['CHA_ESP_drive_mode', 
             'CHA_MO_drive_mode',
             'rain_sensor',
             'kickdown',
             'ESP_speed',
             'seatbelt_rear_m',
            'driving_program',
            'ts_normalized'
             ]

dynamic_context_var = ['avg_irradiation', 'steering_speed', 'temperature_out', 
                       'light_sensor_rear', 'light_sensor_front', 
                       'temperature_in', 'KBI_speed', 
                       'latitude','longitude', 'altitude'] # todo remove these features in the future
if feat_engg:
       cat_static_context_var = ['car_id', 'month', 'weekday', 'hour', 'season', 'seatbelt_codriver', 'seatbelt_rear_l', # categorical static context
                            'seatbelt_rear_r',  'street_category']
else:
       cat_static_context_var = ['car_id', 'month', 'weekday', 'hour', 'seatbelt_codriver', 'seatbelt_rear_l', # categorical static context
                            'seatbelt_rear_r',  'street_category']
dense_static_context_var =  ['distance_driven_benchmark', 'soc', 'time_second']  # dense static context
status_static_context_var = ['ess_status', 'current_drive_mode', 'current_clima_mode', 'current_media_source', # status static context
                     'nav_guidance_status', 'proximity_to_parking_spot', 'phone_status',
                     'bluetooth_connected', 'phone_os',
                     'new_bluetooth_device_to_pair']
#todo i feel street category is higly fluctuating. might be better to ignore
vehicles = ['SEB880','SEB882','SEB883','SEB885','SEB888','SEB889']

In [35]:
context_data = pd.read_csv(combined_context_path, parse_dates=['datetime'], low_memory=False)
context_data_filtered = context_data[context_data['distance_driven'] != 0]
context_data_filtered['distance_driven_benchmark'] = context_data_filtered.groupby('session')['distance_driven'].transform(lambda x: x - x.min())
context_data['distance_driven_benchmark'] = context_data_filtered['distance_driven_benchmark']
context_data['distance_driven_benchmark'].fillna(0, inplace=True)
context_data['car_id'] += 1 
context_data

/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_70158/2038316123.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_data_filtered['distance_driven_benchmark'] = context_data_filtered.groupby('session')['distance_driven'].transform(lambda x: x - x.min())


,Unnamed: 0,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,...,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season,phone_os,new_bluetooth_device_to_pair,distance_driven_benchmark
0,0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair,0.0
1,1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair,0.0
2,2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,0.0,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair,0.0
3,3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair,0.0
4,4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124879,10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,28.5,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair,145.0
10124880,10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,28.5,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair,145.0
10124881,10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,28.5,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair,145.0
10124882,10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,28.5,...,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair,145.0


In [5]:
context_data_list = []
for session in context_data.session.unique().tolist():
    context_data_curr = context_data[context_data['session']== session]
    context_data_curr['distance_driven_benchmark'] = context_data_curr['distance_driven_benchmark'].replace(0, method='ffill')
    context_data_list.append(context_data_curr)
context_data = pd.concat(context_data_list, axis=0)

/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_70158/3471941147.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  context_data_curr['distance_driven_benchmark'] = context_data_curr['distance_driven_benchmark'].replace(0, method='ffill')


In [6]:
context_data

,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,time_second,distance_driven,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,distance_driven_benchmark
0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,...,0.0,0.0,1.0,9.0,2.0,20.0,1.0,1.0,0.0,0.0
1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,...,1.0,10845.0,1.0,9.0,2.0,20.0,1.0,1.0,0.0,0.0
2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,0.0,0.00,...,2.0,10845.0,1.0,9.0,2.0,20.0,1.0,1.0,0.0,0.0
3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,0.00,...,3.0,10845.0,1.0,9.0,2.0,20.0,1.0,1.0,0.0,0.0
4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,0.00,...,3.0,10845.0,1.0,9.0,2.0,20.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,28.5,92.48,...,5360.0,146.0,6.0,3.0,4.0,17.0,1.0,1.0,1.0,145.0
10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,28.5,93.12,...,5361.0,146.0,6.0,3.0,4.0,17.0,1.0,1.0,1.0,145.0
10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,28.5,93.44,...,5362.0,146.0,6.0,3.0,4.0,17.0,1.0,1.0,1.0,145.0
10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,28.5,93.76,...,5363.0,146.0,6.0,3.0,4.0,17.0,1.0,1.0,1.0,145.0


In [7]:

# import matplotlib.pyplot as plt

# # Assuming df is your DataFrame containing the 'session' and 'distance_driven_benchmark' columns

# # Group the DataFrame by 'session'
# grouped_df = context_data.groupby('session')

# # Iterate over each group and create line plots
# for session, group in grouped_df:
#     plt.figure(figsize=(4, 3)) 
#     plt.figure()  # Create a new figure for each session
#     plt.plot(group.index, group['time_second'], marker='o', linestyle='-')
#     plt.title(f'Distance Driven Benchmark for Session {session}')
#     plt.xlabel('Index')
#     plt.ylabel('Distance Driven Benchmark')
#     plt.grid(True)
#     plt.show()


In [8]:
with open(train_session_path, 'rb') as pickle_file:
    train_sessions = pickle.load(pickle_file)
with open(test_session_path, 'rb') as pickle_file:
    test_sessions = pickle.load(pickle_file)
context_data = context_data[context_data['session'].isin(train_sessions + test_sessions)]

train_sequence = pd.read_csv(train_sequence_path, sep='\t', low_memory=False)
test_sequence = pd.read_csv(test_sequence_path, sep='\t', low_memory=False)
selected_sequence = pd.concat([train_sequence, test_sequence], axis=0).sort_values(['session', 'window_id'])

# selected_sequence = pd.read_csv(sequence_context_path, parse_dates=['datetime'], index_col=0)
# selected_sequence['session'] = selected_sequence['session'].astype(int)
# min_datetime_indices = selected_sequence.groupby('session')['datetime'].idxmin()
# selected_sequence = selected_sequence.drop(min_datetime_indices)
# selected_sequence.reset_index(drop=True, inplace=True)

# selected_dfs = []
# for session in tqdm(selected_sequence['session'].unique().tolist()):
#     selected_sequence_curr = selected_sequence[selected_sequence['session']==session]
#     context_data_curr = context_data[context_data['session']==session]
#     context_data_curr = context_data_curr[context_data_curr['datetime']<=selected_sequence_curr['timestamp_target_interaction'].max()]
#     selected_dfs.append(context_data_curr)
# training_sequence_context = pd.concat(selected_dfs, axis=0)
training_sequence_context = context_data

In [9]:
print(len(context_data.session.unique().tolist()), 
      len(training_sequence_context.session.unique().tolist()))

1634 1634


In [10]:
if sequence_augmentation == True:
    augmented_frames = []
    for index, row in tqdm(selected_sequence.iterrows(), total=len(selected_sequence)):
        session = row['session']
        window_id = row['window_id']
        timestamp_target_interaction = row['timestamp_target_interaction']
        training_sequence_context_curr = training_sequence_context[(training_sequence_context['session'] == session) &
                                                    (training_sequence_context['datetime'] <= timestamp_target_interaction)].copy()
        if training_sequence_context_curr.empty:
            print(session, window_id)
        if not whole_session_context and window < len(training_sequence_context_curr):
                training_sequence_context_curr = training_sequence_context_curr.tail(window)
        training_sequence_context_curr['window_id'] = window_id
        augmented_frames.append(training_sequence_context_curr)
        # print(session, window_id, timestamp_target_interaction)
        # break
    training_sequence_context_augmented = pd.concat(augmented_frames, axis=0)
    context_data = training_sequence_context_augmented.reset_index(drop=True)
    context_data['wind_id'] = context_data.groupby(['session', 'window_id']).ngroup()
else:
    # if sequence_augmentation is set to false
    if not whole_session_context:
        context_data = training_sequence_context.groupby('session').tail(window)
    context_data = training_sequence_context.reset_index(drop=True)
    context_data['window_id'] = context_data.groupby('session').ngroup()

100%|██████████| 5883/5883 [03:38<00:00, 26.88it/s]


In [11]:
print('total number of sequence data sessions: ', len(selected_sequence.session.unique().tolist()))
print('total number of Sequence data windows: ', len(train_sequence.window_id.unique().tolist()) + len(test_sequence.window_id.unique().tolist()))
print('total number of context data sessions: ', len(context_data.session.unique().tolist()))
print('total number of context data windows: ', len(context_data.wind_id.unique().tolist()))
#dont be the bothered about the total number of windows.

total number of sequence data sessions:  1634
total number of Sequence data windows:  5883
total number of context data sessions:  1634
total number of context data windows:  5883


In [12]:
# testing_sessions = [16, 25]
# selected_sequence = selected_sequence[selected_sequence['session'].isin(testing_sessions)]
# training_sequence_context = training_sequence_context[training_sequence_context['session'].isin(testing_sessions)]

# window_id = 0
# if sequence_augmentation == True:
#     grouped_selected_sequence = selected_sequence.groupby('session')
#     augmented_frames = []
#     for session, selected_sequence_curr in tqdm(grouped_selected_sequence):
#         for i, row in selected_sequence_curr.iloc[::-1].iterrows():
#             context_filt_curr = training_sequence_context[
#                 (training_sequence_context['session'] == session) &
#                 (training_sequence_context['datetime'] < row['datetime'])].copy()
#             if not whole_session_context:
#                 context_filt_curr = context_filt_curr.tail(window)
#             context_filt_curr['window_id'] = window_id
#             # context_filt_curr['session'] = session
#             augmented_frames.append(context_filt_curr)
#             window_id += 1
#     training_sequence_context_augmented = pd.concat(augmented_frames, axis=0)
#     context_data = training_sequence_context_augmented.reset_index(drop=True)
# else:
#     # if sequence_augmentation is set to false
#     if not whole_session_context:
#         context_data = training_sequence_context.groupby('session').tail(window)
#     context_data = training_sequence_context.reset_index(drop=True)
#     context_data['window_id'] = context_data.groupby('session').ngroup()

In [13]:
dynamic_context = context_data[dynamic_context_var + ['window_id', 'session', 'datetime', 'wind_id']]
print('number of dynamic context session', len(dynamic_context[['window_id', 'session']].drop_duplicates()))

# function to pad first value to fit the window size
if pad_to_window_size:
    df = dynamic_context.copy()
    session_counts = df.groupby('wind_id').size()
    less_than_100 = session_counts[session_counts < window].index.tolist()
    print(f'Number of window with window length less than {window}: ', len(less_than_100))
    window100_dfs = df[~df['wind_id'].isin(less_than_100)]
    empty_df = []
    for window_id in tqdm(less_than_100):
        sub_df = df[df['wind_id'] == window_id]
        rows_to_pad = window - len(sub_df)
        min_datetime_row = sub_df.loc[sub_df['datetime'].idxmin()]
        pad_df = pd.DataFrame(min_datetime_row, df.columns).transpose()
        pad_df = pd.concat([pad_df] * int(rows_to_pad), ignore_index=True, axis=0)

        padded_df = pd.concat([pad_df, sub_df], axis=0).reset_index(drop=True)
        # padded_df['wind_id'] = window_id
        # padded_df['window_id'] = window_id
        # padded_df['wind_id'] = window_id
        empty_df.append(padded_df)
    if empty_df:
        df = pd.concat(empty_df, axis=0).reset_index(drop=True)
        df = pd.concat([df, window100_dfs], axis=0).sort_values(by=['window_id']).reset_index(drop=True)
        session_counts = df.groupby('window_id').size()
        less_than_100 = session_counts[session_counts < window].index.tolist()
        print(f'Number of window with window length less than {window}: ', len(less_than_100))
        dynamic_context = df

number of dynamic context session 5883
Number of window with window length less than 100:  208


100%|██████████| 208/208 [00:01<00:00, 173.57it/s]


Number of window with window length less than 100:  0


In [14]:
if feat_engg:
    dense_static_context = context_data[dense_static_context_var + ['window_id', 'session', 'datetime', 'wind_id']]
    status_static_context = context_data[status_static_context_var + ['window_id', 'session', 'datetime', 'wind_id']]
    for col in status_static_context_var:
        print(col)
        print(sorted(status_static_context[col].unique().tolist()))  
    for col in status_static_context_var:
        status_static_context[col], _ = pd.factorize(status_static_context[col])  
else:
    dense_static_context = context_data[dense_static_context_var + ['window_id', 'session', 'datetime', 'wind_id']]
    
dense_static_context = dense_static_context.sort_values(by=['wind_id','datetime'], ascending=False)
dense_static_context = dense_static_context.groupby('wind_id').first()
dense_static_context.reset_index(inplace=True)
dense_static_context = dense_static_context.sort_values(by='wind_id')
dense_static_context

,wind_id,distance_driven_benchmark,soc,time_second,window_id,session,datetime
0,0,2.0,48.55,3138.0,0,7.0,2022-09-07 21:00:51
1,1,49.0,56.65,3297.0,0,16.0,2022-09-09 18:43:10
2,2,44.0,58.10,2150.0,1,16.0,2022-09-09 18:24:03
3,3,18.0,63.50,846.0,2,16.0,2022-09-09 18:02:19
4,4,0.0,56.65,352.0,1,20.0,2022-09-10 17:15:06
...,...,...,...,...,...,...,...
5878,5878,23.0,56.30,891.0,1258,5387.0,2023-03-28 12:54:10
5879,5879,52.0,8.50,1846.0,4620,5388.0,2023-03-28 14:16:00
5880,5880,125.0,60.85,6216.0,4621,5389.0,2023-03-28 18:38:29
5881,5881,41.0,55.45,2391.0,4622,5399.0,2023-03-31 16:19:44


In [15]:
cat_static_context = context_data[cat_static_context_var + ['window_id', 'session', 'datetime', 'wind_id']]
cat_static_context = cat_static_context.groupby('wind_id').apply(lambda x: x.mode().iloc[0]).reset_index(drop=True)
if feat_engg:
    cat_static_context['season'], _ = pd.factorize(cat_static_context['season'])  
    static_context = pd.merge(cat_static_context, status_static_context, on=['wind_id', 'window_id', 'session'], how='inner')
else:
    static_context = cat_static_context
static_context = static_context.sort_values(by='wind_id')
print('number of windows', len(dynamic_context.wind_id.unique().tolist()), len(static_context.wind_id.unique().tolist()))
print('number of session', len(dynamic_context.session.unique().tolist()), len(static_context.session.unique().tolist()))
static_context

number of windows 5883 5883
number of session 1634 1634


,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,street_category,window_id,session,datetime,wind_id
0,1.0,9.0,2.0,21.0,1.0,1.0,0.0,1.0,0.0,7.0,2022-09-07 20:59:14,0.0
1,1.0,9.0,4.0,18.0,0.0,1.0,0.0,7.0,0.0,16.0,2022-09-09 18:41:36,1.0
2,1.0,9.0,4.0,18.0,0.0,1.0,0.0,4.0,1.0,16.0,2022-09-09 18:22:25,2.0
3,1.0,9.0,4.0,18.0,0.0,1.0,0.0,5.0,2.0,16.0,2022-09-09 18:00:40,3.0
4,1.0,9.0,5.0,17.0,0.0,1.0,0.0,1.0,1.0,20.0,2022-09-10 17:13:27,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5878,6.0,3.0,1.0,12.0,0.0,0.0,0.0,7.0,1258.0,5387.0,2023-03-28 12:52:45,5878.0
5879,6.0,3.0,1.0,14.0,0.0,0.0,0.0,5.0,4620.0,5388.0,2023-03-28 14:07:29,5879.0
5880,6.0,3.0,1.0,18.0,0.0,0.0,0.0,0.0,4621.0,5389.0,2023-03-28 18:37:13,5880.0
5881,6.0,3.0,4.0,16.0,1.0,1.0,1.0,3.0,4622.0,5399.0,2023-03-31 16:19:44,5881.0


In [16]:
dynamic_context

,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,latitude,longitude,altitude,window_id,session,datetime,wind_id
0,0.0,14.890339,20.0,0.0,0.0,26.0,20.442914,48.86775,8.810342,444.0,0,7.0,2022-09-07 20:59:13,0
1,0.0,0.0,20.5,3200.0,768.0,26.5,0.0,48.771829,9.208283,370.0,0,16.0,2022-09-09 18:41:58,1
2,0.0,0.0,20.5,3200.0,774.0,26.5,0.0,48.771828,9.208283,370.0,0,16.0,2022-09-09 18:41:59,1
3,0.0,0.0,20.5,3200.0,774.0,26.5,0.0,48.771827,9.208283,370.0,0,16.0,2022-09-09 18:42:00,1
4,0.0,0.0,20.5,3200.0,780.0,26.5,0.0,48.771826,9.208284,370.0,0,16.0,2022-09-09 18:42:01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588295,37.789256,14.343311,10.0,9600.0,2427.118215,27.5,72.651963,48.572366,9.590398,735.607726,4623,5399.0,2023-03-31 16:07:06,5882
588296,30.0,6.288697,10.0,8426.938647,2561.191841,27.5,79.379413,48.572408,9.590138,733.607536,4623,5399.0,2023-03-31 16:07:05,5882
588297,40.0,33.722877,10.0,8400.0,2532.656724,27.5,82.957043,48.572456,9.589565,729.607809,4623,5399.0,2023-03-31 16:07:03,5882
588298,30.0,0.0,9.5,10431.1668,3788.805012,27.5,60.292066,48.571514,9.591373,745.607303,4623,5399.0,2023-03-31 16:07:14,5882


In [17]:
static_context.columns

Index(['car_id', 'month', 'weekday', 'hour', 'seatbelt_codriver',
       'seatbelt_rear_l', 'seatbelt_rear_r', 'street_category', 'window_id',
       'session', 'datetime', 'wind_id'],
      dtype='object', name=0)

In [18]:
dynamic_context.columns

Index(['avg_irradiation', 'steering_speed', 'temperature_out',
       'light_sensor_rear', 'light_sensor_front', 'temperature_in',
       'KBI_speed', 'latitude', 'longitude', 'altitude', 'window_id',
       'session', 'datetime', 'wind_id'],
      dtype='object')

In [19]:
dense_static_context.columns

Index(['wind_id', 'distance_driven_benchmark', 'soc', 'time_second',
       'window_id', 'session', 'datetime'],
      dtype='object')

In [20]:
dynamic_context = dynamic_context.sort_values(by=['wind_id', 'datetime'])
static_context = cat_static_context.sort_values(by=['wind_id', 'datetime'])
dense_static_context = dense_static_context.sort_values(by=['wind_id', 'datetime'])

dynamic_context = dynamic_context.drop(columns='wind_id')
static_context = cat_static_context.drop(columns=['wind_id'])
dense_static_context = dense_static_context.drop(columns=['wind_id'])

train_dynamic_context = dynamic_context[dynamic_context['session'].isin(train_sessions)].reset_index(drop=True)
test_dynamic_context = dynamic_context[dynamic_context['session'].isin(test_sessions)].reset_index(drop=True)
train_static_context = static_context[static_context['session'].isin(train_sessions)].reset_index(drop=True)
test_static_context = static_context[static_context['session'].isin(test_sessions)].reset_index(drop=True)
train_dense_static_context = dense_static_context[dense_static_context['session'].isin(train_sessions)].reset_index(drop=True)
test_dense_static_context = dense_static_context[dense_static_context['session'].isin(test_sessions)].reset_index(drop=True)

print('number of session', len(train_dynamic_context.window_id.unique().tolist()), len(test_dynamic_context.window_id.unique().tolist()),
       len(train_static_context.window_id.unique().tolist()), len(test_static_context.window_id.unique().tolist()),
       len(train_dense_static_context.window_id.unique().tolist()), len(test_dense_static_context.window_id.unique().tolist()))

train_dynamic_context.to_csv(f'{base_path}{augmentation_folder}dynamic_context/unnormal/train.csv', index=False)
test_dynamic_context.to_csv(f'{base_path}{augmentation_folder}dynamic_context/unnormal/test.csv', index=False)

train_dense_static_context.to_csv(f'{base_path}{augmentation_folder}dense_static_context/unnormal/train.csv', index=False)
test_dense_static_context.to_csv(f'{base_path}{augmentation_folder}dense_static_context/unnormal/test.csv', index=False)

number of session 4624 1259 4624 1259 4624 1259


In [21]:
#Normalisation
dynamic_context_to_normalize = [col for col in train_dynamic_context.columns if col not in ['window_id', 'wind_id', 'session_ids', 'datetime', 'session_id', 'session']]
scaler_dynamic_context = MinMaxScaler()
scaler_dynamic_context.fit(train_dynamic_context[dynamic_context_to_normalize])
train_dynamic_context[dynamic_context_to_normalize] = scaler_dynamic_context.transform(train_dynamic_context[dynamic_context_to_normalize])
test_dynamic_context[dynamic_context_to_normalize] = scaler_dynamic_context.transform(test_dynamic_context[dynamic_context_to_normalize])

scaler_dense_static_context = MinMaxScaler()
scaler_dense_static_context.fit(train_dense_static_context[dense_static_context_var])
train_dense_static_context[dense_static_context_var] = scaler_dense_static_context.transform(train_dense_static_context[dense_static_context_var])
test_dense_static_context[dense_static_context_var] = scaler_dense_static_context.transform(test_dense_static_context[dense_static_context_var])

train_dynamic_context.to_csv(train_dynamic_context_path, index=False)
test_dynamic_context.to_csv(test_dynamic_context_path, index=False)

train_static_context.to_csv(train_static_context_path, index=False)
test_static_context.to_csv(test_static_context_path, index=False)

train_dense_static_context.to_csv(train_dense_static_context_path, index=False)
test_dense_static_context.to_csv(test_dense_static_context_path, index=False)

In [22]:
test_dense_static_context

,distance_driven_benchmark,soc,time_second,window_id,session,datetime
0,0.002403,0.573671,0.069124,0,16.0,2022-09-09 18:43:10
1,0.002158,0.588354,0.045076,1,16.0,2022-09-09 18:24:03
2,0.000883,0.643038,0.017737,2,16.0,2022-09-09 18:02:19
3,0.000294,0.508861,0.020190,3,42.0,2022-09-15 09:43:45
4,0.000294,0.583797,0.027360,4,76.0,2022-09-25 12:00:20
...,...,...,...,...,...,...
1254,0.003384,0.664810,0.082500,1254,5346.0,2023-03-16 09:25:17
1255,0.003286,0.670886,0.081473,1255,5346.0,2023-03-16 09:24:28
1256,0.000834,0.835949,0.052645,1256,5346.0,2023-03-16 09:01:33
1257,0.000392,0.865316,0.047005,1257,5346.0,2023-03-16 08:57:04


In [23]:
test_static_context

,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,street_category,window_id,session,datetime
0,1.0,9.0,4.0,18.0,0.0,1.0,0.0,7.0,0.0,16.0,2022-09-09 18:41:36
1,1.0,9.0,4.0,18.0,0.0,1.0,0.0,4.0,1.0,16.0,2022-09-09 18:22:25
2,1.0,9.0,4.0,18.0,0.0,1.0,0.0,5.0,2.0,16.0,2022-09-09 18:00:40
3,1.0,9.0,3.0,9.0,0.0,0.0,0.0,7.0,3.0,42.0,2022-09-15 09:43:45
4,1.0,9.0,6.0,11.0,1.0,1.0,1.0,2.0,4.0,76.0,2022-09-25 11:58:41
...,...,...,...,...,...,...,...,...,...,...,...
1254,6.0,3.0,3.0,9.0,0.0,0.0,0.0,5.0,1254.0,5346.0,2023-03-16 09:23:38
1255,6.0,3.0,3.0,9.0,0.0,0.0,0.0,5.0,1255.0,5346.0,2023-03-16 09:22:49
1256,6.0,3.0,3.0,9.0,0.0,0.0,0.0,7.0,1256.0,5346.0,2023-03-16 08:59:54
1257,6.0,3.0,3.0,8.0,0.0,0.0,0.0,7.0,1257.0,5346.0,2023-03-16 08:55:30


In [24]:
test_dynamic_context

,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,latitude,longitude,altitude,window_id,session,datetime
0,0.000000,0.000000,0.652174,0.027668,0.122854,0.591837,0.000000,0.930604,0.675377,0.393357,0,16.0,2022-09-09 18:41:30
1,0.000000,0.000000,0.652174,0.027668,0.121875,0.591837,0.000000,0.930604,0.675377,0.393357,0,16.0,2022-09-09 18:41:31
2,0.000000,0.000000,0.652174,0.027668,0.123110,0.591837,0.000000,0.930604,0.675377,0.392585,0,16.0,2022-09-09 18:41:32
3,0.000000,0.000000,0.652174,0.027668,0.121876,0.591837,0.000000,0.930604,0.675377,0.392999,0,16.0,2022-09-09 18:41:33
4,0.000000,0.000000,0.652174,0.027668,0.122429,0.591837,0.000000,0.930604,0.675377,0.393357,0,16.0,2022-09-09 18:41:34
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125895,0.289671,0.000000,0.391304,0.308300,1.000000,0.591837,0.555980,0.931589,0.665911,0.368503,1258,5387.0,2023-03-28 12:54:06
125896,0.295263,0.000000,0.391304,0.308300,1.000000,0.591837,0.555980,0.931594,0.665930,0.368220,1258,5387.0,2023-03-28 12:54:07
125897,0.284907,0.003582,0.391304,0.312253,1.000000,0.591837,0.557252,0.931600,0.665949,0.367374,1258,5387.0,2023-03-28 12:54:08
125898,0.280198,0.000000,0.391304,0.308300,1.000000,0.591837,0.557252,0.931605,0.665969,0.367090,1258,5387.0,2023-03-28 12:54:09


In [25]:
train_static_context

,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,street_category,window_id,session,datetime
0,1.0,9.0,2.0,21.0,1.0,1.0,0.0,1.0,0.0,7.0,2022-09-07 20:59:14
1,1.0,9.0,5.0,17.0,0.0,1.0,0.0,1.0,1.0,20.0,2022-09-10 17:13:27
2,1.0,9.0,6.0,10.0,1.0,1.0,1.0,1.0,2.0,25.0,2022-09-11 10:26:32
3,1.0,9.0,6.0,12.0,1.0,1.0,1.0,7.0,3.0,27.0,2022-09-11 12:24:58
4,1.0,9.0,0.0,17.0,0.0,0.0,0.0,0.0,4.0,33.0,2022-09-12 17:09:43
...,...,...,...,...,...,...,...,...,...,...,...
4619,6.0,3.0,1.0,10.0,0.0,0.0,0.0,7.0,4619.0,5386.0,2023-03-28 10:09:07
4620,6.0,3.0,1.0,14.0,0.0,0.0,0.0,5.0,4620.0,5388.0,2023-03-28 14:07:29
4621,6.0,3.0,1.0,18.0,0.0,0.0,0.0,0.0,4621.0,5389.0,2023-03-28 18:37:13
4622,6.0,3.0,4.0,16.0,1.0,1.0,1.0,3.0,4622.0,5399.0,2023-03-31 16:19:44


In [26]:
train_dense_static_context

,distance_driven_benchmark,soc,time_second,window_id,session,datetime
0,0.000098,0.491646,0.065790,0,7.0,2022-09-07 21:00:51
1,0.000000,0.573671,0.007380,1,20.0,2022-09-10 17:15:06
2,0.000000,0.542785,0.002432,2,25.0,2022-09-11 10:28:01
3,0.000000,0.540253,0.001342,3,27.0,2022-09-11 12:26:02
4,0.000049,0.928608,0.003795,4,33.0,2022-09-12 17:09:43
...,...,...,...,...,...,...
4619,0.000000,0.762025,0.023901,4619,5386.0,2023-03-28 10:10:12
4620,0.002550,0.086076,0.038703,4620,5388.0,2023-03-28 14:16:00
4621,0.006130,0.616203,0.130323,4621,5389.0,2023-03-28 18:38:29
4622,0.002011,0.561519,0.050129,4622,5399.0,2023-03-31 16:19:44


In [27]:
train_dynamic_context

,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,latitude,longitude,altitude,window_id,session,datetime
0,0.000000,0.018372,0.641304,0.000000,0.000000,0.581633,0.081277,0.932435,0.646192,0.434025,0,7.0,2022-09-07 20:59:13
1,0.000000,0.015560,0.641304,0.000000,0.000000,0.581633,0.073700,0.932434,0.646185,0.434025,0,7.0,2022-09-07 20:59:14
2,0.000000,0.015560,0.641304,0.000000,0.000000,0.581633,0.073700,0.932434,0.646185,0.434025,0,7.0,2022-09-07 20:59:14
3,0.000000,0.030418,0.641304,0.000000,0.000000,0.581633,0.064885,0.932433,0.646178,0.434025,0,7.0,2022-09-07 20:59:16
4,0.000000,0.123233,0.641304,0.000000,0.000000,0.584917,0.054707,0.932433,0.646175,0.434025,0,7.0,2022-09-07 20:59:17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
462395,0.030000,0.000000,0.402174,0.055336,0.142997,0.612245,0.155216,0.926702,0.704482,0.588793,4623,5399.0,2023-03-31 16:08:14
462396,0.030000,0.000000,0.402174,0.055623,0.167572,0.612245,0.162231,0.926701,0.704491,0.588793,4623,5399.0,2023-03-31 16:08:15
462397,0.022611,0.125882,0.402174,0.062945,0.289466,0.612245,0.170897,0.926699,0.704494,0.587884,4623,5399.0,2023-03-31 16:08:16
462398,0.012617,0.049353,0.402174,0.067194,0.346719,0.612245,0.195343,0.926697,0.704494,0.587663,4623,5399.0,2023-03-31 16:08:17


In [28]:
def session_window_mapping(df):
    session_window_dict = {}

    for index, row in df.iterrows():
        session = int(row['session'])
        window_id = int(row['window_id'])
        
        # If the session is not already in the dictionary, initialize an empty set
        if session not in session_window_dict:
            session_window_dict[session] = set()
        
        # Add the window_id to the set corresponding to the session
        session_window_dict[session].add(window_id)

    # Convert sets to lists in the resulting dictionary
    session_window_dict = {session: list(window_ids) for session, window_ids in session_window_dict.items()}
    return session_window_dict

train_session_win_id_mapping_dc = session_window_mapping(train_dynamic_context)
test_session_win_id_mapping_dc = session_window_mapping(test_dynamic_context)
train_session_win_id_mapping_sc = session_window_mapping(train_static_context)
test_session_win_id_mapping_sc = session_window_mapping(test_static_context)
train_session_win_id_mapping_dsc = session_window_mapping(train_dense_static_context)
test_session_win_id_mapping_dsc = session_window_mapping(test_dense_static_context)

with open(os.path.join(parameter_path, 'session_win_id_mapping.pkl'), 'rb') as pickle_file:
    train_session_win_id_mapping = pickle.load(pickle_file)
    test_session_win_id_mapping = pickle.load(pickle_file)

In [29]:
print(len(train_session_win_id_mapping_dc), len(train_session_win_id_mapping_sc),  len(train_session_win_id_mapping_dsc), len(train_session_win_id_mapping))

1308 1308 1308 1308


In [30]:
if train_session_win_id_mapping_dc == train_session_win_id_mapping_sc == train_session_win_id_mapping == train_session_win_id_mapping_dsc:
    print("All training data mapping are exactly identical.")
if test_session_win_id_mapping_dc == test_session_win_id_mapping_sc == test_session_win_id_mapping == test_session_win_id_mapping_dsc:
    print("All testing data mapping are exactly identical.")

All training data mapping are exactly identical.
All testing data mapping are exactly identical.


In [31]:
print(len(train_dynamic_context.session.unique().tolist()), len(test_dynamic_context.session.unique().tolist()))
print(len(train_dynamic_context.window_id.unique().tolist()), len(test_dynamic_context.window_id.unique().tolist()))
print(len(train_static_context.session.unique().tolist()), len(test_static_context.session.unique().tolist()))
print(len(train_static_context.window_id.unique().tolist()), len(test_static_context.window_id.unique().tolist()))
print(len(train_dense_static_context.session.unique().tolist()), len(test_dense_static_context.session.unique().tolist()))
print(len(train_dense_static_context.window_id.unique().tolist()), len(test_dense_static_context.window_id.unique().tolist()))

1308 326
4624 1259
1308 326
4624 1259
1308 326
4624 1259
